In [2]:


import org.apache.spark.sql.functions._

val spark = SparkSession.builder()
.appName("StreetTreeCensus")
.getOrCreate()

val treeDF = spark.read.format("csv").option("header","true").load("/host/Users/projects/juypterNotebook/DisneyStreaming/2015StreetTreesCensus_TREES.csv")
treeDF.coalesce(4)
treeDF.cache



// NOTE: add minimum and maximum values to thresholds
val thresholds: Array[Double] = Array(Double.MinValue, 0.0) ++ (((0.0 until 50.0 by 10).toArray ++ Array(Double.MaxValue)).map(_.toDouble))

// Convert DataFrame to RDD and calculate histogram values
val _tmpHist = treeDF.select($"tree_dbh" cast "double").rdd.map(r => r.getDouble(0)).histogram(thresholds)

// Result DataFrame contains `from`, `to` range and the `value`.
val histogram = sc.parallelize((thresholds, thresholds.tail, _tmpHist).zipped.toList).toDF("from", "to", "value")



[Stage 1:======                                                     (0 + 4) / 4]========>                                            (1 + 3) / 4]

spark = org.apache.spark.sql.SparkSession@3de841d4
treeDF = [created_at: string, tree_id: string ... 40 more fields]
thresholds = Array(-1.7976931348623157E308, 0.0, 0.0, 10.0, 20.0, 30.0, 40.0, 1.7976931348623157E308)
_tmpHist = Array(0, 17932, 328456, 224856, 84112, 25322, 3110)
histogram = [from: double, to: double ... 1 more field]


[from: double, to: double ... 1 more field]